# CS253 PYTHON ASSIGNMENT
This notebook contains experimentation with 'doctor' characteristic.

## IMPORTING LIBRARIES AND CREATING DF

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [29]:
df=pd.read_csv('../train.csv')

In [30]:
def convert_to_numeric(value):
    if isinstance(value, str):
        if 'Crore' in value:
            numeric_value = float(value.replace(' Crore+', '')) * 1000000000
        elif 'Lac' in value:
            numeric_value = float(value.replace(' Lac+', '')) * 10000000
        elif 'Thou' in value:
            numeric_value = float(value.replace(' Thou+', '')) * 1000
        elif 'Hund' in value:
            numeric_value = float(value.replace(' Hund+', '')) * 100
        else:
            numeric_value = float(value)
    else:
        numeric_value = float(value)
    return numeric_value

In [31]:
df['Total Assets'] = df['Total Assets'].apply(convert_to_numeric)
df['Liabilities'] = df['Liabilities'].apply(convert_to_numeric)
le=LabelEncoder()
df['state']=le.fit_transform(df['state'])
le_1=LabelEncoder()
df['Party']=le_1.fit_transform(df['Party'])
le_2=LabelEncoder()
df['Education']=le_2.fit_transform(df['Education'])

In [32]:
# create doctor columm
df['Doctor'] = 0
for i in range(len(df)):
    if 'Dr.' in df['Candidate'][i]:
        df['Doctor'][i] = 1
    else:
        df['Doctor'][i] = 0

/var/folders/g8/wtrpgc1d0qs7tht6k4b437vw0000gn/T/ipykernel_72967/1239247916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Doctor'][i] = 0
/var/folders/g8/wtrpgc1d0qs7tht6k4b437vw0000gn/T/ipykernel_72967/1239247916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Doctor'][i] = 0
/var/folders/g8/wtrpgc1d0qs7tht6k4b437vw0000gn/T/ipykernel_72967/1239247916.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Doctor'][i]

# TRAIN TEST SPLIT

In [33]:
X=df.drop(['Candidate','ID','Education','Constituency ∇'],axis=1)
y=df['Education']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# DECISION TREE CLASSIFIER

In [34]:
param_grid = {'max_depth':range(1,20),'min_samples_split':[2,3,4,5,6,7,8,9,10],'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10]}
dt_model = DecisionTreeClassifier()
dt_model_cv = GridSearchCV(dt_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
dt_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 20),
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='f1_weighted')

In [35]:
print('Best parameters:',dt_model_cv.best_params_)
y_pred=dt_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('F1 Score:',f1_score(y_test,y_pred,average='weighted'))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))

Best parameters: {'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 8}
Accuracy: 0.2111650485436893
F1 Score: 0.18905649273233366
Precision: 0.18143690344234462
Recall: 0.2111650485436893


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# KNN

In [36]:
param_grid={'n_neighbors':range(1,40)}
knn_model=KNeighborsClassifier()
knn_model_cv=GridSearchCV(knn_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
knn_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 40)}, scoring='f1_weighted')

In [37]:
print('Best parameters:',knn_model_cv.best_params_)
y_pred=knn_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('F1 Score:',f1_score(y_test,y_pred,average='weighted'))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))

Best parameters: {'n_neighbors': 1}
Accuracy: 0.17718446601941748
F1 Score: 0.17628375703531926
Precision: 0.1764198523780389
Recall: 0.17718446601941748


# LINEAR SVC

In [38]:
param_grid={'C':[0.1,1,10,100,1000],'gamma':[1,0.1,0.01,0.001,0.0001],'kernel':['rbf']}
svm_model=SVC()
svm_model_cv=GridSearchCV(svm_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
svm_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='f1_weighted')

In [39]:
print('Best parameters:',svm_model_cv.best_params_)
y_pred=svm_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('F1 Score:',f1_score(y_test,y_pred,average='weighted'))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))

Best parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 0.19174757281553398
F1 Score: 0.15259486863594737
Precision: 0.15347545488066694
Recall: 0.19174757281553398


# RANDOM FOREST CLASSIFIER

In [40]:
param_grid={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
rf_model=RandomForestClassifier()
rf_model_cv=GridSearchCV(rf_model,param_grid=param_grid,cv=10,scoring='f1_weighted')
rf_model_cv.fit(X_train,y_train)

/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1440 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1440 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/aditikhandelia/anaconda3/lib/python3.11/

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 10, None],
                         'max_features': [1, 3, 5, 7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [10, 100, 200]},
             scoring='f1_weighted')

In [41]:
print('Best Parameters:',rf_model_cv.best_params_)
y_pred=rf_model_cv.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred))
print('Precision:',precision_score(y_test,y_pred,average='weighted'))
print('Recall:',recall_score(y_test,y_pred,average='weighted'))
print('F1:',f1_score(y_test,y_pred,average='weighted'))

Best Parameters: {'max_depth': 10, 'max_features': 7, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 100}
Accuracy: 0.22815533980582525
Precision: 0.1960611169579431
Recall: 0.22815533980582525
F1: 0.1948080627216462


/Users/aditikhandelia/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## RESULTS

In all the four models, the final F1 score was lesser than the base F1 score of 0.24.